# Import бибилиотек

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install scikit-survival

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [4]:
import pandas as pd
import numpy as np
from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [5]:
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [6]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.7 MB/s eta 0:00:00


In [7]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

# Основные данные

## Набор данных по клиентам

In [8]:
clients = pd.read_csv('/content/drive/MyDrive/соревнования/data-fusion2024-churn/clients.csv')
clients

,user_id,report,employee_count_nm,bankemplstatus,customer_age
0,3,2,ОТ 101 ДО 500,0,3
1,9,1,БОЛЕЕ 1001,0,3
2,13,6,ОТ 501 ДО 1000,0,2
3,37,5,БОЛЕЕ 1001,0,2
4,41,1,ОТ 101 ДО 500,0,2
...,...,...,...,...,...
95995,562043,12,NaN,0,2
95996,562205,12,NaN,0,1
95997,562312,12,NaN,0,0
95998,562721,12,NaN,0,2


In [9]:
clients.duplicated().sum()

0

In [10]:
clients.isnull().sum()

user_id                  0
report                   0
employee_count_nm    36466
bankemplstatus           0
customer_age             0
dtype: int64

In [11]:
clients['employee_count_nm'] = clients['employee_count_nm'].fillna(0)

In [12]:
# делаем ohe кодирование
clients = pd.get_dummies(clients, columns=['employee_count_nm', 'customer_age'])
clients

,user_id,report,bankemplstatus,employee_count_nm_0,employee_count_nm_БОЛЕЕ 1001,employee_count_nm_БОЛЕЕ 500,employee_count_nm_ДО 10,employee_count_nm_ОТ 101 ДО 500,employee_count_nm_ОТ 11 ДО 30,employee_count_nm_ОТ 11 ДО 50,employee_count_nm_ОТ 31 ДО 50,employee_count_nm_ОТ 501 ДО 1000,employee_count_nm_ОТ 51 ДО 100,customer_age_0,customer_age_1,customer_age_2,customer_age_3
0,3,2,0,False,False,False,False,True,False,False,False,False,False,False,False,False,True
1,9,1,0,False,True,False,False,False,False,False,False,False,False,False,False,False,True
2,13,6,0,False,False,False,False,False,False,False,False,True,False,False,False,True,False
3,37,5,0,False,True,False,False,False,False,False,False,False,False,False,False,True,False
4,41,1,0,False,False,False,False,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,12,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False
95996,562205,12,0,True,False,False,False,False,False,False,False,False,False,False,True,False,False
95997,562312,12,0,True,False,False,False,False,False,False,False,False,False,True,False,False,False
95998,562721,12,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False


## Набор данных по отчетам

In [13]:
report_dates = pd.read_csv('/content/drive/MyDrive/соревнования/data-fusion2024-churn/report_dates.csv',parse_dates=['report_dt'])
report_dates

,report,report_dt
0,1,2022-07-31 03:00:00
1,2,2022-08-31 03:00:00
2,3,2022-09-30 03:00:00
3,4,2022-10-31 03:00:00
4,5,2022-11-30 03:00:00
5,6,2022-12-31 03:00:00
6,7,2023-01-31 03:00:00
7,8,2023-02-28 03:00:00
8,9,2023-03-31 03:00:00
9,10,2023-04-30 03:00:00


## Набор данных по транзакциям

In [14]:
transactions = pd.read_csv('/content/drive/MyDrive/соревнования/data-fusion2024-churn/transactions.csv',parse_dates=['transaction_dttm'])
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,3,3,1,-183.883957,2022-01-28 12:05:33
1,3,3,1,-3206.437012,2022-01-28 12:52:30
2,3,16,1,-153866.890625,2022-02-16 14:45:56
3,3,56,1,-15144.601562,2022-03-09 19:58:29
4,3,0,1,5297.908691,2022-03-12 18:11:31
...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49


In [15]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13075023 entries, 0 to 13075022
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   user_id           int64         
 1   mcc_code          int64         
 2   currency_rk       int64         
 3   transaction_amt   float64       
 4   transaction_dttm  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 498.8 MB


In [16]:
transactions.isnull().sum()

user_id             0
mcc_code            0
currency_rk         0
transaction_amt     0
transaction_dttm    0
dtype: int64

## Объединение датафреймов

In [17]:
clients=merge_by_concat(clients, report_dates, ['report'])
clients

,user_id,report,bankemplstatus,employee_count_nm_0,employee_count_nm_БОЛЕЕ 1001,employee_count_nm_БОЛЕЕ 500,employee_count_nm_ДО 10,employee_count_nm_ОТ 101 ДО 500,employee_count_nm_ОТ 11 ДО 30,employee_count_nm_ОТ 11 ДО 50,employee_count_nm_ОТ 31 ДО 50,employee_count_nm_ОТ 501 ДО 1000,employee_count_nm_ОТ 51 ДО 100,customer_age_0,customer_age_1,customer_age_2,customer_age_3,report_dt
0,3,2,0,False,False,False,False,True,False,False,False,False,False,False,False,False,True,2022-08-31 03:00:00
1,9,1,0,False,True,False,False,False,False,False,False,False,False,False,False,False,True,2022-07-31 03:00:00
2,13,6,0,False,False,False,False,False,False,False,False,True,False,False,False,True,False,2022-12-31 03:00:00
3,37,5,0,False,True,False,False,False,False,False,False,False,False,False,False,True,False,2022-11-30 03:00:00
4,41,1,0,False,False,False,False,True,False,False,False,False,False,False,False,True,False,2022-07-31 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,12,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False,2023-06-30 03:00:00
95996,562205,12,0,True,False,False,False,False,False,False,False,False,False,False,True,False,False,2023-06-30 03:00:00
95997,562312,12,0,True,False,False,False,False,False,False,False,False,False,True,False,False,False,2023-06-30 03:00:00
95998,562721,12,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False,2023-06-30 03:00:00


In [18]:
transactions=merge_by_concat(transactions, clients[['user_id','report_dt']], ['user_id'])
transactions['diff_days'] = (transactions['report_dt']-transactions['transaction_dttm']) / np.timedelta64(1, 'D')
transactions

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm,report_dt,diff_days
0,3,3,1,-183.883957,2022-01-28 12:05:33,2022-08-31 03:00:00,214.621146
1,3,3,1,-3206.437012,2022-01-28 12:52:30,2022-08-31 03:00:00,214.588542
2,3,16,1,-153866.890625,2022-02-16 14:45:56,2022-08-31 03:00:00,195.509769
3,3,56,1,-15144.601562,2022-03-09 19:58:29,2022-08-31 03:00:00,174.292720
4,3,0,1,5297.908691,2022-03-12 18:11:31,2022-08-31 03:00:00,171.367002
...,...,...,...,...,...,...,...
13075018,562740,155,1,-2484.366211,2023-03-20 11:52:09,2023-06-30 03:00:00,101.630451
13075019,562740,9,1,-187.658463,2023-03-20 12:10:22,2023-06-30 03:00:00,101.617801
13075020,562740,1,1,-891.933350,2023-03-20 15:53:37,2023-06-30 03:00:00,101.462766
13075021,562740,13,1,-464.467316,2023-03-20 15:54:49,2023-06-30 03:00:00,101.461933


In [19]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13075023 entries, 0 to 13075022
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   user_id           int64         
 1   mcc_code          int64         
 2   currency_rk       int64         
 3   transaction_amt   float64       
 4   transaction_dttm  datetime64[ns]
 5   report_dt         datetime64[ns]
 6   diff_days         float64       
dtypes: datetime64[ns](2), float64(2), int64(3)
memory usage: 698.3 MB


# Новые признаки

In [20]:
agg_transations = transactions.groupby(['user_id'], as_index=False).agg(
    count_amount = ('transaction_amt', 'count'), # количество транзакций
    min_amount = ('transaction_amt', 'min'), # минимальные транзакции
    max_amount = ('transaction_amt', 'max'), # максимальные транзакции
    mean_amount = ('transaction_amt', 'mean'), # средняя транзакция
    sum_amount=('transaction_amt', 'sum'), # сумма всех транзакций
    median_amount=('transaction_amt', 'median'), # медианное значение транзакций
    std_amount=('transaction_amt', 'std'), # средне квад отклонение транзакций
    neg_trans_rate=('transaction_amt', lambda x: (x < 0).mean()), # доля отрицательных транзакций
    pos_trans_rate=('transaction_amt', lambda x: (x >= 0).mean()), # доля положительных транзакций
    neg_count_trans=('transaction_amt', lambda x: (x < 0).sum()), # количество отрицательных транзакций
    pos_count_trans=('transaction_amt', lambda x: (x >= 0).sum()), # количество положительных транзакций
    sum_neg_trans=('transaction_amt', lambda x: x[x < 0].sum()), # сумма положительных транзакций
    sum_pos_trans=('transaction_amt', lambda x: x[x >= 0].sum()), # сумма отрицательных транзакций
    min_time_diff=('transaction_dttm', lambda x: x.diff().min().total_seconds() if len(x) > 1 else np.nan), # минимальная разница между транзакциями
    max_time_diff=('transaction_dttm', lambda x: x.diff().max().total_seconds() if len(x) > 1 else np.nan), # максимальная разница между транзакциями
    mean_time_diff=('transaction_dttm', lambda x: x.diff().mean().total_seconds() if len(x) > 1 else np.nan), # средняя разница между транзакциями
    unique_days=('transaction_dttm', lambda x: x.dt.date.nunique()), #
    weekend_trans_rate=('transaction_dttm', lambda x: (x.dt.dayofweek >= 5).mean()), # доля транзакций в выходные
    weekday_trans_rate=('transaction_dttm', lambda x: (x.dt.dayofweek < 5).mean()) # доля транзакций в будни
)

agg_transations

,user_id,count_amount,min_amount,max_amount,mean_amount,sum_amount,median_amount,std_amount,neg_trans_rate,pos_trans_rate,neg_count_trans,pos_count_trans,sum_neg_trans,sum_pos_trans,min_time_diff,max_time_diff,mean_time_diff,unique_days,weekend_trans_rate,weekday_trans_rate
0,3,11,-153866.890625,104011.960938,1246.037876,13706.416641,4549.455078,60860.880477,0.363636,0.636364,4,7,-172401.813156,186108.229797,309.0,2601415.0,918925.200000,8,0.272727,0.727273
1,9,90,-90147.617188,-45.579891,-3593.718520,-323434.666813,-840.509674,11797.055119,1.000000,0.000000,90,0,-323434.666813,0.000000,2.0,1884276.0,176001.247191,54,0.422222,0.577778
2,13,22,-58740.300781,70322.828125,-5668.971780,-124717.379150,-10529.004883,23507.953991,0.636364,0.363636,14,8,-253484.063477,128766.684326,1583.0,1867993.0,692674.952380,18,0.136364,0.863636
3,37,315,-35782.984375,5487.140625,-1053.522538,-331859.599463,-236.420776,3913.114835,0.993651,0.006349,313,2,-342598.388037,10738.788574,7.0,598372.0,49259.703821,129,0.276190,0.723810
4,41,16,-16841.208984,-290.766998,-6786.663385,-108586.614166,-6328.293701,5251.737982,1.000000,0.000000,16,0,-108586.614166,0.000000,109.0,2692933.0,882473.066666,12,0.062500,0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,37,-13020.519531,-40.769005,-799.493409,-29581.256115,-239.170517,2136.038913,1.000000,0.000000,37,0,-29581.256115,0.000000,411.0,4616164.0,296939.861111,23,0.243243,0.756757
95996,562205,151,-6220.171387,1595.461060,-268.158335,-40491.908630,-147.305283,590.682233,0.980132,0.019868,148,3,-42460.814964,1968.906334,33.0,695880.0,102729.620000,85,0.364238,0.635762
95997,562312,56,-1372.377075,-28.292030,-331.032523,-18537.821270,-271.973526,309.587795,1.000000,0.000000,56,0,-18537.821270,0.000000,272.0,3203127.0,248176.945454,39,0.339286,0.660714
95998,562721,85,-18981.269531,5412.773926,-1929.467785,-164004.761685,-399.343048,4172.623511,0.776471,0.223529,66,19,-198395.925579,34391.163893,272.0,2197939.0,179881.059523,55,0.129412,0.870588


## Доп признаки

In [21]:
# Максимальная дата до отчета, по клиентам
max_day_trans = transactions.groupby(['user_id'])[['diff_days']].max().reset_index().\
rename(columns={"diff_days":"max_day_trans"}).sort_values(["user_id"]).reset_index(drop=True)

In [22]:
# Минимальная дата до отчета, по клиентам
min_day_trans = transactions.groupby(['user_id'])[['diff_days']].min().reset_index().\
rename(columns={"diff_days":"min_day_trans"}).sort_values(["user_id"]).reset_index(drop=True)

In [23]:
# Средняя дата до отчета, по клиентам
mean_day_trans = transactions.groupby(['user_id'])[['diff_days']].mean().reset_index().\
rename(columns={"diff_days":"mean_day_trans"}).sort_values(["user_id"]).reset_index(drop=True)

In [24]:
# Сумма дней до отчета, по клиентам
sum_day_trans = transactions.groupby(['user_id'])[['diff_days']].sum().reset_index().\
rename(columns={"diff_days":"sum_day_trans"}).sort_values(["user_id"]).reset_index(drop=True)

In [25]:
# Среднее время суток совершения транзакций
transactions['hour_of_day'] = transactions['transaction_dttm'].dt.hour
average_hour_of_day = transactions.groupby('user_id')['hour_of_day'].mean().reset_index().\
rename(columns={"hour_of_day":"average_hour_of_day"}).sort_values(["user_id"]).reset_index(drop=True)

In [26]:
threshold = transactions['transaction_amt'].quantile(0.95)
threshold

105.73154220580975

In [27]:
# Доля крупных транзакций
transactions['is_large_transaction'] = (transactions['transaction_amt'] > threshold).astype(int)
large_transactions = transactions.groupby('user_id')['is_large_transaction'].mean().reset_index().\
rename(columns={"is_large_transaction":"large_transactions"}).sort_values(["user_id"]).reset_index(drop=True)

In [28]:
# Длительность активности
activity_duration = transactions.groupby('user_id')['transaction_dttm'].apply(lambda x: (x.max() - x.min()).days).reset_index().\
rename(columns={"transaction_dttm":"activity_duration"}).sort_values(["user_id"]).reset_index(drop=True)

In [29]:
# Частота использования различных mcc_code
mcc_code_usage = transactions.groupby('user_id')['mcc_code'].nunique().reset_index().\
rename(columns={"mcc_code":"mcc_code_usage"}).sort_values(["user_id"]).reset_index(drop=True)

In [30]:
#Поведение клиента в день недели
preferred_weekday_for_transactions = transactions.groupby('user_id')['transaction_dttm'].apply(lambda x: x.dt.dayofweek.mode()[0]).reset_index().\
rename(columns={"transaction_dttm":"preferred_weekday_for_transactions"}).sort_values(["user_id"]).reset_index(drop=True)

In [31]:
#Суммарное количество уникальных валют клиента
unique_currencies_count = transactions.groupby('user_id')['currency_rk'].nunique().reset_index().\
rename(columns={"currency_rk":"unique_currencies_count"}).sort_values(["user_id"]).reset_index(drop=True)

## Итоговый с признаками

In [32]:
clients=merge_by_concat(clients, agg_transations, ['user_id'])

In [33]:
clients=merge_by_concat(clients, max_day_trans, ['user_id'])

In [34]:
clients=merge_by_concat(clients, min_day_trans, ['user_id'])

In [35]:
clients=merge_by_concat(clients, mean_day_trans, ['user_id'])

In [36]:
clients=merge_by_concat(clients, sum_day_trans, ['user_id'])

In [37]:
clients=merge_by_concat(clients, average_hour_of_day, ['user_id'])

In [38]:
clients=merge_by_concat(clients, large_transactions, ['user_id'])

In [39]:
clients=merge_by_concat(clients, activity_duration, ['user_id'])

In [40]:
clients=merge_by_concat(clients, mcc_code_usage, ['user_id'])

In [41]:
clients=merge_by_concat(clients, preferred_weekday_for_transactions, ['user_id'])

In [42]:
clients=merge_by_concat(clients, unique_currencies_count, ['user_id'])

In [43]:
clients

,user_id,report,bankemplstatus,employee_count_nm_0,employee_count_nm_БОЛЕЕ 1001,employee_count_nm_БОЛЕЕ 500,employee_count_nm_ДО 10,employee_count_nm_ОТ 101 ДО 500,employee_count_nm_ОТ 11 ДО 30,employee_count_nm_ОТ 11 ДО 50,...,max_day_trans,min_day_trans,mean_day_trans,sum_day_trans,average_hour_of_day,large_transactions,activity_duration,mcc_code_usage,preferred_weekday_for_transactions,unique_currencies_count
0,3,2,0,False,False,False,False,True,False,False,...,214.621146,108.264062,155.459226,1710.051481,17.545455,0.636364,106,4,4,1
1,9,1,0,False,True,False,False,False,False,False,...,283.587488,102.289907,198.650554,17878.549850,11.711111,0.000000,181,22,5,1
2,13,6,0,False,False,False,False,False,False,False,...,282.616204,114.257708,202.556765,4456.248819,14.227273,0.363636,168,4,4,2
3,37,5,0,False,True,False,False,False,False,False,...,283.736042,104.713507,187.245439,58982.313275,12.012698,0.006349,179,28,2,1
4,41,1,0,False,False,False,False,True,False,False,...,256.469896,103.262766,170.750356,2732.005694,17.437500,0.000000,153,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,12,0,True,False,False,False,False,False,False,...,266.598877,142.873935,225.177718,8331.575579,15.702703,0.000000,123,12,2,1
95996,562205,12,0,True,False,False,False,False,False,False,...,280.398981,102.048947,188.307409,28434.418819,16.589404,0.013245,178,15,4,1
95997,562312,12,0,True,False,False,False,False,False,False,...,280.244144,122.261134,210.175032,11769.801817,16.321429,0.000000,157,7,5,1
95998,562721,12,0,True,False,False,False,False,False,False,...,280.883727,105.999363,211.173836,17949.776100,8.529412,0.200000,174,11,4,1


## Работа с train

In [44]:
train = pd.read_csv('/content/drive/MyDrive/соревнования/data-fusion2024-churn/train.csv')
train

,user_id,target,time
0,3,0,77
1,13,0,86
2,37,0,89
3,41,0,57
4,42,0,84
...,...,...,...
63995,561824,0,91
63996,562043,0,75
63997,562312,0,91
63998,562721,0,29


In [45]:
clients.columns

Index(['user_id', 'report', 'bankemplstatus', 'employee_count_nm_0',
       'employee_count_nm_БОЛЕЕ 1001', 'employee_count_nm_БОЛЕЕ 500',
       'employee_count_nm_ДО 10', 'employee_count_nm_ОТ 101 ДО 500',
       'employee_count_nm_ОТ 11 ДО 30', 'employee_count_nm_ОТ 11 ДО 50',
       'employee_count_nm_ОТ 31 ДО 50', 'employee_count_nm_ОТ 501 ДО 1000',
       'employee_count_nm_ОТ 51 ДО 100', 'customer_age_0', 'customer_age_1',
       'customer_age_2', 'customer_age_3', 'report_dt', 'count_amount',
       'min_amount', 'max_amount', 'mean_amount', 'sum_amount',
       'median_amount', 'std_amount', 'neg_trans_rate', 'pos_trans_rate',
       'neg_count_trans', 'pos_count_trans', 'sum_neg_trans', 'sum_pos_trans',
       'min_time_diff', 'max_time_diff', 'mean_time_diff', 'unique_days',
       'weekend_trans_rate', 'weekday_trans_rate', 'max_day_trans',
       'min_day_trans', 'mean_day_trans', 'sum_day_trans',
       'average_hour_of_day', 'large_transactions', 'activity_duration',
   

In [46]:
clients_columns = ['user_id',
                  'bankemplstatus',
                  'employee_count_nm_0',
                  'employee_count_nm_БОЛЕЕ 1001',
                  'employee_count_nm_БОЛЕЕ 500',
                  'employee_count_nm_ДО 10',
                  'employee_count_nm_ОТ 101 ДО 500',
                  'employee_count_nm_ОТ 11 ДО 30',
                  'employee_count_nm_ОТ 11 ДО 50',
                  'employee_count_nm_ОТ 31 ДО 50',
                  'employee_count_nm_ОТ 501 ДО 1000',
                  'employee_count_nm_ОТ 51 ДО 100',
                  'customer_age_0',
                  'customer_age_1',
                  'customer_age_2',
                  'customer_age_3',
                  'count_amount',
                  'min_amount',
                  'max_amount',
                  'mean_amount',
                  'sum_amount',
                  'median_amount',
                  'std_amount',
                  'neg_trans_rate',
                  'pos_trans_rate',
                  'neg_count_trans',
                  'pos_count_trans',
                  'sum_neg_trans',
                  'sum_pos_trans',
                  'min_time_diff',
                  'max_time_diff',
                  'mean_time_diff',
                  'unique_days',
                  'weekend_trans_rate',
                  'weekday_trans_rate',
                  'max_day_trans',
                  'min_day_trans',
                  'mean_day_trans',
                  'sum_day_trans',
                  'average_hour_of_day',
                  'large_transactions',
                  'activity_duration',
                  'mcc_code_usage',
                  'preferred_weekday_for_transactions',
                  'unique_currencies_count']

In [47]:
train_columns = ['bankemplstatus',
                  'employee_count_nm_0',
                  'employee_count_nm_БОЛЕЕ 1001',
                  'employee_count_nm_БОЛЕЕ 500',
                  'employee_count_nm_ДО 10',
                  'employee_count_nm_ОТ 101 ДО 500',
                  'employee_count_nm_ОТ 11 ДО 30',
                  'employee_count_nm_ОТ 11 ДО 50',
                  'employee_count_nm_ОТ 31 ДО 50',
                  'employee_count_nm_ОТ 501 ДО 1000',
                  'employee_count_nm_ОТ 51 ДО 100',
                  'customer_age_0',
                  'customer_age_1',
                  'customer_age_2',
                  'customer_age_3',
                  'count_amount',
                  'min_amount',
                  'max_amount',
                  'mean_amount',
                  'sum_amount',
                  'median_amount',
                  'std_amount',
                  'neg_trans_rate',
                  'pos_trans_rate',
                  'neg_count_trans',
                  'pos_count_trans',
                  'sum_neg_trans',
                  'sum_pos_trans',
                  'min_time_diff',
                  'max_time_diff',
                  'mean_time_diff',
                  'unique_days',
                  'weekend_trans_rate',
                  'weekday_trans_rate',
                  'max_day_trans',
                  'min_day_trans',
                  'mean_day_trans',
                  'sum_day_trans',
                  'average_hour_of_day',
                  'large_transactions',
                  'activity_duration',
                  'mcc_code_usage',
                  'preferred_weekday_for_transactions',
                  'unique_currencies_count',
]

In [48]:
df = merge_by_concat(clients, train, ['user_id'])
df

,user_id,report,bankemplstatus,employee_count_nm_0,employee_count_nm_БОЛЕЕ 1001,employee_count_nm_БОЛЕЕ 500,employee_count_nm_ДО 10,employee_count_nm_ОТ 101 ДО 500,employee_count_nm_ОТ 11 ДО 30,employee_count_nm_ОТ 11 ДО 50,...,mean_day_trans,sum_day_trans,average_hour_of_day,large_transactions,activity_duration,mcc_code_usage,preferred_weekday_for_transactions,unique_currencies_count,target,time
0,3,2,0,False,False,False,False,True,False,False,...,155.459226,1710.051481,17.545455,0.636364,106,4,4,1,0.0,77.0
1,9,1,0,False,True,False,False,False,False,False,...,198.650554,17878.549850,11.711111,0.000000,181,22,5,1,NaN,NaN
2,13,6,0,False,False,False,False,False,False,False,...,202.556765,4456.248819,14.227273,0.363636,168,4,4,2,0.0,86.0
3,37,5,0,False,True,False,False,False,False,False,...,187.245439,58982.313275,12.012698,0.006349,179,28,2,1,0.0,89.0
4,41,1,0,False,False,False,False,True,False,False,...,170.750356,2732.005694,17.437500,0.000000,153,5,0,1,0.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,12,0,True,False,False,False,False,False,False,...,225.177718,8331.575579,15.702703,0.000000,123,12,2,1,0.0,75.0
95996,562205,12,0,True,False,False,False,False,False,False,...,188.307409,28434.418819,16.589404,0.013245,178,15,4,1,NaN,NaN
95997,562312,12,0,True,False,False,False,False,False,False,...,210.175032,11769.801817,16.321429,0.000000,157,7,5,1,0.0,91.0
95998,562721,12,0,True,False,False,False,False,False,False,...,211.173836,17949.776100,8.529412,0.200000,174,11,4,1,0.0,29.0


In [49]:
df['time'] = df['time'].fillna(-1)
df['time'] = df['time'].astype(np.int32)

In [50]:
df['target'] = df['target'].fillna(-1)
df['target'] = df['target'].astype(np.int8)

In [51]:
missing_values = df.isnull().sum()
sorted_missing_values = missing_values.sort_values()

In [52]:
df = df.fillna(0)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

## Кластеризация

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Создаем несколько новых признаков с разным числом кластеров
n_clusters_list = [5, 10, 20]

for n_clusters in n_clusters_list:

    # Применяем K-means для n_clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df[f'cluster_label_{n_clusters}'] = kmeans.fit_predict(df)

In [ ]:
df

# Обучение

In [ ]:
X = df[df['time']!=-1][train_columns].copy()
y = df[df['time']!=-1][['target', 'time']].copy()

In [ ]:
y['target'] = y['target'].astype(bool)

In [ ]:
aux = [(e1,e2) for e1,e2 in np.array(y)]

In [ ]:
y = np.array(aux, dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train

array([(False, 91.), (False, 90.), (False, 67.), ..., (False, 91.),
       (False, 91.), (False, 91.)],
      dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

## Пробуем RandomSurvivalForest

In [ ]:
rsf = RandomSurvivalForest(
    n_estimators=350, max_depth=12, min_samples_split=15, min_samples_leaf=5, n_jobs=-1, random_state=42,
    verbose=2, low_memory=True
)
rsf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 350building tree 2 of 350

building tree 3 of 350
building tree 4 of 350
building tree 5 of 350
building tree 6 of 350
building tree 7 of 350
building tree 8 of 350
building tree 9 of 350
building tree 10 of 350
building tree 11 of 350
building tree 12 of 350
building tree 13 of 350
building tree 14 of 350
building tree 15 of 350
building tree 16 of 350
building tree 17 of 350
building tree 18 of 350
building tree 19 of 350
building tree 20 of 350
building tree 21 of 350
building tree 22 of 350
building tree 23 of 350
building tree 24 of 350
building tree 25 of 350
building tree 26 of 350
building tree 27 of 350
building tree 28 of 350
building tree 29 of 350
building tree 30 of 350
building tree 31 of 350
building tree 32 of 350
building tree 33 of 350
building tree 34 of 350
building tree 35 of 350
building tree 36 of 350
building tree 37 of 350
building tree 38 of 350


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 33.1min


building tree 39 of 350
building tree 40 of 350
building tree 41 of 350
building tree 42 of 350
building tree 43 of 350
building tree 44 of 350
building tree 45 of 350
building tree 46 of 350
building tree 47 of 350
building tree 48 of 350
building tree 49 of 350
building tree 50 of 350
building tree 51 of 350
building tree 52 of 350
building tree 53 of 350
building tree 54 of 350
building tree 55 of 350
building tree 56 of 350
building tree 57 of 350
building tree 58 of 350
building tree 59 of 350
building tree 60 of 350
building tree 61 of 350
building tree 62 of 350
building tree 63 of 350
building tree 64 of 350
building tree 65 of 350
building tree 66 of 350
building tree 67 of 350
building tree 68 of 350
building tree 69 of 350
building tree 70 of 350
building tree 71 of 350
building tree 72 of 350
building tree 73 of 350
building tree 74 of 350
building tree 75 of 350
building tree 76 of 350
building tree 77 of 350
building tree 78 of 350
building tree 79 of 350
building tree 80

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 141.1min


building tree 160 of 350
building tree 161 of 350
building tree 162 of 350
building tree 163 of 350
building tree 164 of 350
building tree 165 of 350
building tree 166 of 350
building tree 167 of 350
building tree 168 of 350
building tree 169 of 350
building tree 170 of 350
building tree 171 of 350
building tree 172 of 350
building tree 173 of 350
building tree 174 of 350
building tree 175 of 350
building tree 176 of 350
building tree 177 of 350
building tree 178 of 350
building tree 179 of 350
building tree 180 of 350
building tree 181 of 350
building tree 182 of 350
building tree 183 of 350
building tree 184 of 350
building tree 185 of 350
building tree 186 of 350
building tree 187 of 350
building tree 188 of 350
building tree 189 of 350
building tree 190 of 350
building tree 191 of 350
building tree 192 of 350
building tree 193 of 350
building tree 194 of 350
building tree 195 of 350
building tree 196 of 350
building tree 197 of 350
building tree 198 of 350
building tree 199 of 350


[Parallel(n_jobs=-1)]: Done 350 out of 350 | elapsed: 310.1min finished


RandomSurvivalForest(low_memory=True, max_depth=12, min_samples_leaf=5,
                     min_samples_split=15, n_estimators=350, n_jobs=-1,
                     random_state=42, verbose=2)

## Метрики модели

In [ ]:
rsf.score(X_test, y_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 350 out of 350 | elapsed:    0.9s finished


0.7637905384431884

# Решение

In [ ]:
X_pred = df[df['time']==-1][train_columns].copy()
X_pred

,bankemplstatus,employee_count_nm_0,employee_count_nm_БОЛЕЕ 1001,employee_count_nm_БОЛЕЕ 500,employee_count_nm_ДО 10,employee_count_nm_ОТ 101 ДО 500,employee_count_nm_ОТ 11 ДО 30,employee_count_nm_ОТ 11 ДО 50,employee_count_nm_ОТ 31 ДО 50,employee_count_nm_ОТ 501 ДО 1000,...,negative_general_trans_info_1000_median,negative_general_trans_info_1000_median,negative_general_trans_info_1000_median,negative_general_trans_info_1000_median,num_transaction_before_5_days,num_transaction_last_5_days,percent_last_5,num_transaction_before_30_days,num_transaction_last_30_days,percent_last_30
1,0,0,1,0,0,0,0,0,0,0,...,-840.509674,-840.509674,-840.509674,-840.509674,88.0,2.0,0.022727,79.0,11.0,0.139241
9,0,0,0,0,0,0,0,1,0,0,...,-550.682404,-550.682404,-550.682404,-550.682404,67.0,0.0,0.000000,60.0,7.0,0.116667
10,0,0,0,0,1,0,0,0,0,0,...,-116.574402,-116.574402,-116.574402,-116.574402,11.0,1.0,0.090909,10.0,2.0,0.200000
17,0,0,1,0,0,0,0,0,0,0,...,-772.497192,-772.497192,-772.497192,-772.497192,413.0,14.0,0.033898,292.0,135.0,0.462329
20,0,1,0,0,0,0,0,0,0,0,...,-301.121384,-301.121384,-301.121384,-301.121384,50.0,0.0,0.000000,50.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95988,0,1,0,0,0,0,0,0,0,0,...,-467.705963,-467.705963,-467.705963,-467.705963,99.0,0.0,0.000000,85.0,14.0,0.164706
95990,0,1,0,0,0,0,0,0,0,0,...,-542.066284,-542.066284,-542.066284,-542.066284,72.0,0.0,0.000000,66.0,6.0,0.090909
95993,0,1,0,0,0,0,0,0,0,0,...,-1422.766357,-1422.766357,-1422.766357,-1422.766357,39.0,0.0,0.000000,20.0,19.0,0.950000
95994,0,1,0,0,0,0,0,0,0,0,...,-315.139267,-315.139267,-315.139267,-315.139267,44.0,12.0,0.272727,36.0,20.0,0.555556


In [ ]:
predictions = rsf.predict(X_pred)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done 350 out of 350 | elapsed:    3.0s finished


In [ ]:
predictions

array([1.70952312, 1.75777951, 3.2663236 , ..., 3.83774014, 2.17439319,
       3.6938723 ])

In [ ]:
submit = df[df['time']==-1][['user_id']].copy()

In [ ]:
submit['predict'] = predictions

In [ ]:
submit

,user_id,predict
1,9,1.709523
9,61,1.757780
10,62,3.266324
17,80,0.668853
20,88,11.173808
...,...,...
95988,561362,3.124967
95990,561419,3.374706
95993,561895,3.837740
95994,561908,2.174393


In [ ]:
submit.to_csv('submission.csv',index=False)